# Crypto usage of qmrExchange

### Import required libraries

In [1]:
import sys
sys.path.append('..')
from main import *  

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
time_interval = 'minute'
tickers = ['XYZ']
episodes = 1000

### Instantiate a Simulator

In [3]:
sim = Simulator(time_unit=time_interval, episodes=episodes)
# create a blockchain with mempool for the exchange
sim.exchange.crypto = True
sim.exchange.create_asset(tickers[0])

print(sim.exchange.blockchain.chain)

### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = CryptoMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = CryptoMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

print(sim.agents)

[<Agent: market_maker>, <Agent: market_taker>]


### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [6]:
# print(sim.exchange.mempool.transactions)
print(sim.transactions.head(20).to_markdown())

| dt                         | ticker    |         fee |   amount | sender       | recipient    | confirmed   | timestamp           |
|:---------------------------|:----------|------------:|---------:|:-------------|:-------------|:------------|:--------------------|
| 2023-05-19 10:44:09.489042 | init_seed | 0           |     0    | init_seed    | init_seed    | True        | NaT                 |
| 2022-01-01 00:00:00        | XYZ       | 0           |   100    | init_seed    | init_seed    | True        | 2022-01-01 00:01:00 |
| 2022-01-01 00:01:00        | XYZ       | 0.001       |   100.25 | market_maker | market_taker | True        | 2022-01-01 00:02:00 |
| 2022-01-01 00:02:00        | XYZ       | 0.000333333 |     0    | market_taker | market_maker | True        | 2022-01-01 00:03:00 |
| 2022-01-01 00:03:00        | XYZ       | 0.000333333 |   100.25 | market_maker | market_taker | True        | 2022-01-01 00:05:00 |
| 2022-01-01 00:04:00        | XYZ       | 0.000333333 |   100

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

| dt                  |   open |   high |   low |   close |   volume |
|:--------------------|-------:|-------:|------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 101.25 | 99    |   99    |       12 |
| 2022-01-01 00:15:00 |  98.75 |  98.75 | 98    |   98.49 |       23 |
| 2022-01-01 00:30:00 |  98.74 |  98.74 | 97.99 |   98.48 |       15 |
| 2022-01-01 00:45:00 |  98.23 |  98.71 | 97.97 |   98.71 |       16 |
| 2022-01-01 01:00:00 |  98.96 |  98.96 | 98.21 |   98.21 |       13 |
| 2022-01-01 01:15:00 |  97.96 |  97.96 | 97    |   97.48 |       27 |
| 2022-01-01 01:30:00 |  97.72 |  97.72 | 97.48 |   97.72 |        6 |
| 2022-01-01 01:45:00 |  97.96 |  97.96 | 97.48 |   97.48 |        9 |
| 2022-01-01 02:00:00 |  97.24 |  97.24 | 97    |   97    |       13 |
| 2022-01-01 02:15:00 |  97.24 |  97.24 | 96.52 |   96.52 |       18 |
| 2022-01-01 02:30:00 |  96.28 |  96.76 | 96.28 |   96.76 |       11 |
| 2022-01-01 02:45:00 |  97    |  97    | 96.28 |   96.76 |       20 |
| 2022

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

|                     |              XYZ |      cash |              aum |
|:--------------------|-----------------:|----------:|-----------------:|
| 2022-01-01 00:00:00 |      0           |     0     |      0           |
| 2022-01-01 00:01:00 |      0           |     0     |      0           |
| 2022-01-01 00:02:00 |  10025           |  -100.251 |   9924.75        |
| 2022-01-01 00:03:00 |  20100.1         |  -200.502 |  19899.6         |
| 2022-01-01 00:04:00 |  30225.4         |  -300.753 |  29924.6         |
| 2022-01-01 00:05:00 |  50375.6         |  -501.254 |  49874.4         |
| 2022-01-01 00:06:00 |  80826.7         |  -802.256 |  80024.4         |
| 2022-01-01 00:07:00 | 121604           | -1204.01  | 120400           |
| 2022-01-01 00:08:00 | 162181           | -1605.76  | 160575           |
| 2022-01-01 00:09:00 | 202758           | -2007.51  | 200750           |
| 2022-01-01 00:10:00 | 243334           | -2409.26  | 240925           |
| 2022-01-01 00:11:00 | 283911        

### Plot the results

In [9]:
from source.backend.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

In [10]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()